# **Dependancies**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import *
from keras.callbacks import *
import os
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from commons import mean_absolute_percentage_error
from keras.layers import *
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


# **Loading Data**

In [2]:
PATH_TO_DATA = "notebooks/datasets/reg_seven.csv"
data = pd.read_csv(PATH_TO_DATA)

In [3]:
data.head()

,difficulty30rsi,difficulty90var,fee_to_reward90rsiUSD,hashrate30var,median_transaction_fee7rocUSD,mining_profitability,price30smaUSD,price3wmaUSD,price7wmaUSD,sentinusd90emaUSD,size90trx,top100cap,transactionvalueUSD,priceUSD
0,97.889,1.168546e+12,49.775,9.266842e+25,36.980,7220.0,56.481,91.616,88.156,68391477,0.334,19.962,2592.0,94.715
1,97.889,1.243755e+12,50.838,9.644142e+25,50.270,7990.0,58.991,100.288,93.861,73007549,0.331,20.024,4400.0,107.749
2,97.889,1.315554e+12,51.407,1.004435e+26,51.448,8852.0,61.879,111.703,101.342,77645623,0.329,19.987,4478.0,120.003
3,97.889,1.383943e+12,52.170,9.733186e+25,44.093,8777.0,64.777,119.740,108.257,82154725,0.327,19.990,4249.0,123.562
4,98.186,1.468007e+12,50.982,9.145296e+25,48.589,9950.0,67.766,125.520,115.097,86170134,0.326,19.833,4348.0,128.664


In [4]:
data.shape

(2466, 14)

In [5]:
X = data.iloc[:,:-1]

In [6]:
y=data.iloc[:,-1:]

In [7]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, train_size=0.8, shuffle=True, random_state=7)

In [8]:
X_train.shape

(1972, 13)

In [9]:
estimators=[]

In [10]:
estimators.append(['robust',RobustScaler()])

In [11]:
estimators.append(['mixmax',MinMaxScaler()])

In [12]:
scale=Pipeline(estimators,verbose=True)

In [13]:
X_train=scale.fit_transform(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


In [14]:
X_test=scale.transform(X_test)

In [15]:
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))

In [16]:
X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

In [17]:
y_train=y_train.values

In [18]:
y_train=np.reshape(y_train, (y_train.shape[0],1,1))

In [19]:
y_test=y_test.values

In [20]:
y_test=np.reshape(y_test,(y_test.shape[0],1,1))

# **Model Architecture + Training**

In [22]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [23]:
adam=optimizers.adam(lr=lr_schedule(0),amsgrad=True)


Learning rate:  0.001


In [24]:
model = Sequential()
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu')))
model.add(Dense(1))
model.compile(loss="logcosh", optimizer=adam, metrics=['mae'])

In [25]:
mcp_save = ModelCheckpoint('trained_models/LSTM_reg_seven_new.hdf5', save_best_only=True, monitor='val_loss', mode='auto')
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')

In [26]:
model.fit(X_train,y_train, epochs=5000, batch_size=32, validation_data=(X_test,y_test), callbacks=[mcp_save,earlyStopping])

Instructions for updating:
Use tf.cast instead.
Train on 1972 samples, validate on 494 samples
Epoch 1/5000
1972/1972 [==============================] - 6s 3ms/step - loss: 2971.6613 - mae: 2972.3540 - val_loss: 2952.9614 - val_mae: 2953.6545
Epoch 2/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 2509.6507 - mae: 2510.3433 - val_loss: 2384.3314 - val_mae: 2385.0247
Epoch 3/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 1873.5358 - mae: 1874.2291 - val_loss: 1128.6280 - val_mae: 1129.3213
Epoch 4/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 638.3509 - mae: 639.0435 - val_loss: 499.6657 - val_mae: 500.3587
Epoch 5/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 442.3088 - mae: 442.9995 - val_loss: 424.0711 - val_mae: 424.7608
Epoch 6/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 381.6811 - mae: 382.3708 - val_loss: 348.3630 - val_mae: 349.0516
Epoch 7/5000
1972/1972 [=====

Epoch 56/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 85.5631 - mae: 86.2421 - val_loss: 86.5778 - val_mae: 87.2610
Epoch 57/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 82.5259 - mae: 83.2056 - val_loss: 84.3335 - val_mae: 85.0097
Epoch 58/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 85.1028 - mae: 85.7822 - val_loss: 80.1287 - val_mae: 80.8017
Epoch 59/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 83.3269 - mae: 84.0016 - val_loss: 92.9185 - val_mae: 93.6047
Epoch 60/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 85.1113 - mae: 85.7911 - val_loss: 83.6543 - val_mae: 84.3341
Epoch 61/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 82.2403 - mae: 82.9165 - val_loss: 80.4554 - val_mae: 81.1303
Epoch 62/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 81.5240 - mae: 82.2023 - val_loss: 79.3284 - val_mae: 80.0066
Epoch 63/5000

1972/1972 [==============================] - 5s 2ms/step - loss: 67.6710 - mae: 68.3444 - val_loss: 66.3836 - val_mae: 67.0694
Epoch 115/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 66.6277 - mae: 67.3004 - val_loss: 61.5330 - val_mae: 62.2052
Epoch 116/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 72.0082 - mae: 72.6908 - val_loss: 71.5091 - val_mae: 72.1973
Epoch 117/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 67.8308 - mae: 68.5030 - val_loss: 61.6767 - val_mae: 62.3550
Epoch 118/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 66.5342 - mae: 67.2111 - val_loss: 69.0474 - val_mae: 69.7250
Epoch 119/5000
1972/1972 [==============================] - 5s 2ms/step - loss: 68.4048 - mae: 69.0798 - val_loss: 68.7217 - val_mae: 69.4100
Epoch 120/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 66.7777 - mae: 67.4543 - val_loss: 60.3682 - val_mae: 61.0383
Epoch 121/5000
1972/1

Epoch 172/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 61.8372 - mae: 62.5105 - val_loss: 58.3634 - val_mae: 59.0455
Epoch 173/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 62.7328 - mae: 63.4049 - val_loss: 56.6611 - val_mae: 57.3228
Epoch 174/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 65.8253 - mae: 66.5024 - val_loss: 56.6734 - val_mae: 57.3496
Epoch 175/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 63.0754 - mae: 63.7512 - val_loss: 57.1366 - val_mae: 57.8176
Epoch 176/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 62.8332 - mae: 63.5120 - val_loss: 61.8902 - val_mae: 62.5773
Epoch 177/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 64.0912 - mae: 64.7687 - val_loss: 57.5802 - val_mae: 58.2643
Epoch 178/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 62.5174 - mae: 63.1902 - val_loss: 64.5061 - val_mae: 65.1841
Epoch 

Epoch 230/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 61.7863 - mae: 62.4659 - val_loss: 59.4315 - val_mae: 60.1062
Epoch 231/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 58.2062 - mae: 58.8718 - val_loss: 59.9719 - val_mae: 60.6376
Epoch 232/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 60.2707 - mae: 60.9428 - val_loss: 53.1964 - val_mae: 53.8719
Epoch 233/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 59.1422 - mae: 59.8166 - val_loss: 52.7507 - val_mae: 53.4135
Epoch 234/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 58.4029 - mae: 59.0768 - val_loss: 57.6240 - val_mae: 58.3044
Epoch 235/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 60.5470 - mae: 61.2198 - val_loss: 53.3551 - val_mae: 54.0227
Epoch 236/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 59.0666 - mae: 59.7393 - val_loss: 54.1528 - val_mae: 54.8294
Epoch 

...

Epoch 636/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 46.4653 - mae: 47.1412 - val_loss: 47.4771 - val_mae: 48.1658
Epoch 637/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 47.4443 - mae: 48.1212 - val_loss: 38.7390 - val_mae: 39.4070
Epoch 638/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 44.1919 - mae: 44.8562 - val_loss: 38.9221 - val_mae: 39.5926
Epoch 639/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 44.7656 - mae: 45.4346 - val_loss: 54.4843 - val_mae: 55.1711
Epoch 640/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 46.6929 - mae: 47.3684 - val_loss: 40.8752 - val_mae: 41.5543
Epoch 641/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 46.4003 - mae: 47.0795 - val_loss: 41.8466 - val_mae: 42.5063
Epoch 642/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 43.6784 - mae: 44.3413 - val_loss: 39.9965 - val_mae: 40.6536
Epoch 

Epoch 694/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 46.0787 - mae: 46.7534 - val_loss: 40.6863 - val_mae: 41.3669
Epoch 695/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 43.4064 - mae: 44.0736 - val_loss: 37.9251 - val_mae: 38.5955
Epoch 696/5000
1972/1972 [==============================] - 5s 2ms/step - loss: 43.7342 - mae: 44.4002 - val_loss: 54.5090 - val_mae: 55.1888
Epoch 697/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 50.1996 - mae: 50.8827 - val_loss: 45.7916 - val_mae: 46.4812
Epoch 698/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 42.3376 - mae: 42.9972 - val_loss: 39.9008 - val_mae: 40.5793
Epoch 699/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 43.1285 - mae: 43.7972 - val_loss: 37.4345 - val_mae: 38.1075
Epoch 700/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 43.9801 - mae: 44.6551 - val_loss: 40.2816 - val_mae: 40.9386
Epoch 

Epoch 752/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 44.9307 - mae: 45.6041 - val_loss: 38.3367 - val_mae: 39.0185
Epoch 753/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 42.2367 - mae: 42.9037 - val_loss: 40.4201 - val_mae: 41.1046
Epoch 754/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 41.7508 - mae: 42.4191 - val_loss: 42.1592 - val_mae: 42.8478
Epoch 755/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 43.0403 - mae: 43.7089 - val_loss: 37.5503 - val_mae: 38.2007
Epoch 756/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 43.1961 - mae: 43.8647 - val_loss: 42.1782 - val_mae: 42.8608
Epoch 757/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 42.6054 - mae: 43.2655 - val_loss: 36.1692 - val_mae: 36.8350
Epoch 758/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 43.3288 - mae: 43.9971 - val_loss: 37.3099 - val_mae: 37.9878
Epoch 

Epoch 810/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 41.1079 - mae: 41.7738 - val_loss: 35.8458 - val_mae: 36.5008
Epoch 811/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 42.8184 - mae: 43.4885 - val_loss: 35.3052 - val_mae: 35.9620
Epoch 812/5000
1972/1972 [==============================] - 5s 2ms/step - loss: 41.9522 - mae: 42.6164 - val_loss: 38.1482 - val_mae: 38.8078
Epoch 813/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 41.8787 - mae: 42.5464 - val_loss: 39.7933 - val_mae: 40.4773
Epoch 814/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 43.5391 - mae: 44.2113 - val_loss: 36.7511 - val_mae: 37.4226
Epoch 815/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 40.9824 - mae: 41.6444 - val_loss: 35.0550 - val_mae: 35.7160
Epoch 816/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 41.4116 - mae: 42.0738 - val_loss: 40.7415 - val_mae: 41.4280
Epoch 

Epoch 868/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 41.6397 - mae: 42.3028 - val_loss: 43.8004 - val_mae: 44.4782
Epoch 869/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 43.5863 - mae: 44.2588 - val_loss: 37.6535 - val_mae: 38.3201
Epoch 870/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 42.0083 - mae: 42.6714 - val_loss: 37.2240 - val_mae: 37.9068
Epoch 871/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 42.3013 - mae: 42.9706 - val_loss: 35.8164 - val_mae: 36.4836
Epoch 872/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 41.7534 - mae: 42.4284 - val_loss: 35.0614 - val_mae: 35.7259
Epoch 873/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 41.7894 - mae: 42.4600 - val_loss: 37.0938 - val_mae: 37.7483
Epoch 874/5000
1972/1972 [==============================] - 4s 2ms/step - loss: 44.9300 - mae: 45.6035 - val_loss: 37.2859 - val_mae: 37.9617
Epoch 

# **Testing Model**

In [27]:
from tensorflow.keras.models import load_model
prediction_model = load_model('trained_models/LSTM_reg_seven_new.hdf5',compile=False)

In [28]:
y_pred = np.ravel(prediction_model.predict(X_test))

In [29]:
y_test=np.ravel(y_test)

In [30]:
r2=r2_score(y_test,y_pred) #testing score/ r^2
r2

0.9995161692423405

In [31]:
mae=mean_absolute_error(y_test,y_pred) #mae
mae

35.716042185424314

In [32]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred)) #rmse
rmse

83.31697071794044

In [33]:
mape=mean_absolute_percentage_error(y_test,y_pred) #mape
mape

1.5003317888342855

In [34]:
pd.DataFrame(zip(['MAE','RMSE','MAPE','R^2'],[mae,rmse,mape,r2])).transpose()

,0,1,2,3
0,MAE,RMSE,MAPE,R^2
1,35.716,83.317,1.50033,0.999516


In [35]:
pd.DataFrame([y_test,y_pred]).transpose()

,0,1
0,6912.000,6930.309570
1,6233.000,6215.681641
2,235.065,236.442825
3,609.126,614.928223
4,453.641,455.907837
5,852.382,832.241394
6,342.284,338.540771
7,344.899,347.315247
8,644.487,628.036255
9,3804.000,3817.390625
